In [ ]:
from typing import Dict, List, Optional
from collections import Counter
import os
import csv
!pip install torchmetrics
!pip install pytorch-metric-learning
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
!pip install pytorch-lightning
import torch.optim as optim
import torchmetrics
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 9.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.3/109.3 KB 4.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.6 MB/s eta 0:00:00


## Todo Part 1
Complete the implementation of the encode method of the Tokenizer class:

`encode`: encode a given space-separated text into list of token ids according to the `self.token2idx` property. For tokens not present in the mapping, use the id of the `<unk>` token. If `max_length` is set, pad the input to `max_length` if it is less than `max_length` and truncate to `max_length` if it exceeds the length.

Examples
```python
text = "hello transformers !"
tokenizer.encode(text)                  # example output: [3, 4, 5]
tokenizer.encode(text, max_length=5)    # example output: [3, 4, 5, 0, 0]
tokenizer.encode(text, max_length=2)    # example output: [3, 4]
```

In [ ]:
class Tokenizer:
    def __init__(self):
        # two special tokens for padding and unknown
        self.token2idx = {"<pad>": 0, "<unk>": 1}
        self.idx2token = ["<pad>", "<unk>"]
        self.is_fit = False
    
    @property
    def pad_id(self):
        return self.token2idx["<pad>"]
    
    def __len__(self):
        return len(self.idx2token)
    
    def fit(self, train_texts: List[str]):
        counter = Counter()
        for text in train_texts:
            counter.update(text.lower().split())
        
        # manually set a vocabulary size for the data set
        vocab_size = 20000
        self.idx2token.extend([token for token, count in counter.most_common(vocab_size - 2)])
        for (i, token) in enumerate(self.idx2token):
            self.token2idx[token] = i
            
        self.is_fit = True
    """            
    def encode(self, text: str, max_length: Optional[int] = None) -> List[int]:
        if not self.is_fit:
            raise Exception("Please fit the tokenizer on the training tokens")
            
        # TODO: implement the encode method, the method signature shouldn't be changed
        raise NotImplemented
    """
    def encode(self, text: str, max_length: Optional[int] = None) -> List[int]:
        if not self.is_fit:
            raise Exception("Please fit the tokenizer on the training tokens")
            
        tokens = text.lower().split()
        if max_length is not None:
            tokens = tokens[:max_length]
        
        token_ids = [self.token2idx.get(token, self.token2idx["<unk>"]) for token in tokens]
        if max_length is not None and len(token_ids) < max_length:
            token_ids += [self.pad_id] * (max_length - len(token_ids))
        
        return token_ids

    def decode(self, token_ids: List[int]) -> str:
        tokens = [self.idx2token[token_id] for token_id in token_ids]
        text = " ".join(tokens)#.replace("<pad>", "").replace("<unk>", "")
        return text.strip()


In [ ]:
def load_raw_data(filepath: str, with_tags: bool = True):
    data = {'text': []}
    if with_tags:
        data['tags'] = []
        with open(filepath) as f:
            reader = csv.reader(f)
            for text, tags in reader:
                data['text'].append(text)
                data['tags'].append(tags)
    else:
        with open(filepath) as f:
            for line in f:
                data['text'].append(line.strip())
    return data

In [ ]:
# from google.colab import files
# train = files.upload()

In [ ]:
# test_tokens = files.upload()

In [ ]:
# val = files.upload()

In [ ]:
tokenizer = Tokenizer()
data_dir = "./"
train_raw = load_raw_data(os.path.join(data_dir, "train.csv"))
val_raw = load_raw_data(os.path.join(data_dir, "val.csv"))
test_raw = load_raw_data(os.path.join(data_dir, "test_tokens.txt"), with_tags=False)
# fit the tokenizer on the training tokens
tokenizer.fit(train_raw['text'])

In [ ]:
#upload the dataset
#for google colb, use this
#from google.colab import files
#uploaded = files.upload()

In [ ]:
class NERDataset: 
    tag2idx = {'O': 1, 'B-PER': 2, 'I-PER': 3, 'B-ORG': 4, 'I-ORG': 5, 'B-LOC': 6, 'I-LOC': 7, 'B-MISC': 8, 'I-MISC': 9}
    idx2tag = ['<pad>', 'O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG','B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
  
    def __init__(self, raw_data: Dict[str, List[str]], tokenizer: Tokenizer, max_length: int = 128):
        self.tokenizer = tokenizer
        self.token_ids = []
        self.tag_ids = []
        self.with_tags = False
        for text in raw_data['text']:
            self.token_ids.append(tokenizer.encode(text, max_length=max_length))
        if 'tags' in raw_data:
            self.with_tags = True
            for tags in raw_data['tags']:
                self.tag_ids.append(self.encode_tags(tags, max_length=max_length))
    
    def encode_tags(self, tags: str, max_length: Optional[int] = None):
        tag_ids = [self.tag2idx[tag] for tag in tags.split()]
        if max_length is None:
            return tag_ids
        # truncate the tags if longer than max_length
        if len(tag_ids) > max_length:
            return tag_ids[:max_length]
        # pad with 0s if shorter than max_length
        else:
            return tag_ids + [0] * (max_length - len(tag_ids))  # 0 as padding for tags
        
    def __len__(self):
        return len(self.token_ids)
    
    def __getitem__(self, idx):
        token_ids = torch.LongTensor(self.token_ids[idx])
        mask = token_ids == self.tokenizer.pad_id  # padding tokens
        if self.with_tags:
            # for training and validation
            return token_ids, mask, torch.LongTensor(self.tag_ids[idx])
        else:
            # for testing
            return token_ids, mask
        

In [ ]:
tr_data = NERDataset(train_raw, tokenizer)
va_data = NERDataset(val_raw, tokenizer)
te_data = NERDataset(test_raw, tokenizer)

In [ ]:
test1 = "my last name is marks i am from san jose california"
test2 = tokenizer.decode(tokenizer.encode(test1))
print(test2)
test3 = "gonna fly now"
print(tokenizer.encode(test3, max_length = 5))
print(tokenizer.decode(tokenizer.encode(test3)))
test4 = """Look I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance six minutes six minutes
Something s wrong I can feel it six minutes Slim Shady you re on
Just a feeling I ve got like something s about to happen but I don t know what
If that means what I think it means  we re in trouble big trouble
And if he is as bananas as you say I'm not taking any chances
You are just what the doc ordered"""
test5 = tokenizer.encode(test4)
print(test5)
test6 = tokenizer.decode(test5)
print(test6)

my last name is marks i am from san jose california
[1, 2200, 237, 0, 0]
<unk> fly now
[1673, 65, 20, 1, 367, 3812, 13, 256, 39, 7, 2321, 1360, 1, 37, 1, 143, 446, 7, 442, 61, 71, 1096, 148, 299, 148, 299, 1799, 4657, 8029, 65, 309, 1485, 28, 148, 299, 9101, 1, 256, 7151, 13, 176, 8, 4857, 65, 1, 851, 541, 1799, 4657, 94, 7, 3289, 37, 65, 6491, 1, 925, 313, 141, 25, 2116, 313, 65, 628, 28, 2116, 69, 7151, 6, 3107, 1011, 3107, 9, 141, 26, 30, 32, 1, 32, 256, 377, 1, 39, 469, 220, 3545, 256, 57, 176, 313, 2, 1, 1155]
look i was <unk> go easy on you not to hurt your <unk> but <unk> only going to get this one chance six minutes six minutes something s wrong i can feel it six minutes slim <unk> you re on just a feeling i <unk> got like something s about to happen but i don <unk> know what if that means what i think it means we re in trouble big trouble and if he is as <unk> as you say <unk> not taking any chances you are just what the <unk> ordered


## Todo Part 2
Implement and experiment with transformer models. The implementation should include **at least** the following:
- `nn.Embedding` layer to embed input token ids to the embedding space
- `nn.TransformerEncoder` layer to perform transformer operations
- `nn.Linear` layer as the output layer to map the output to the number of classes

As we will be using the cross-entropy loss, an `nn.Softmax` or `nn.LogSoftmax` layer is not needed.

You can refer to the following links for transformer Docs and examples:

https://pytorch.org/docs/stable/_modules/torch/nn/modules/transformer.html

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

You can modify the `__init__` method including the signature needed. For the `forward` method, the method signature is given as follows:

- `src`: a `torch.LongTensor` of shape (batch_size, max_length, vocab_size) representing the input text tokens.

- `src_mask`: a `torch.BoolTensor` of shape (batch_size, max_length) indicating whether an input position is padded. This is needed to prevent the transformer model attending to padded tokens.

The output from the `forward` method should be of shape (batch_size, max_length, num_classes). Note that the number of classes should be 10 instead of 9 because of an additional padding class.


In [46]:
# TODO: implement the Transformer model architecture and forward method
# class TransformerModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#     def forward(self, src: torch.Tensor, src_mask: torch.Tensor) -> torch.Tensor:
#         raise NotImplemented

class TransformerModel(nn.Module):
    def __init__(self, vocab_size: int, embed_size: int, num_heads: int, hidden_size: int, num_layers: int, num_classes = 10):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_size, nhead=num_heads, dim_feedforward=hidden_size)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_layer = nn.Linear(embed_size, num_classes)
        
    def forward(self, src: torch.Tensor, src_mask: torch.Tensor) -> torch.Tensor:
        x = self.embedding(src) * torch.sqrt(torch.tensor(self.embedding.embedding_dim, dtype=torch.float32))
        x = x.transpose(0, 1)
        x = self.encoder(x, src_key_padding_mask=src_mask)
        x = x.transpose(0, 1)
        x = self.output_layer(x)
        return x


In [45]:
#modify as required
def validate(
    model: nn.Module, 
    dataloader: DataLoader, 
    device: torch.device,
):
    acc_metric = torchmetrics.Accuracy(task = 'multiclass', num_classes = 10, compute_on_step=False).to(device)
    loss_metric = torchmetrics.MeanMetric(compute_on_step=False).to(device)
    model.eval()
    
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids, input_mask, tags = batch[0].to(device), batch[1].to(device), batch[2].to(device)
            # output shape: (batch_size, max_length, num_classes)
            logits = model(input_ids, input_mask)
            # ignore padding index 0 when calculating loss
            loss = F.cross_entropy(logits.reshape(-1, 10), tags.reshape(-1), ignore_index=0)
                
            loss_metric.update(loss, input_mask.numel() - input_mask.sum())
            is_active = torch.logical_not(input_mask)  # non-padding elements
            # only consider non-padded tokens when calculating accuracy
            acc_metric.update(logits[is_active], tags[is_active])
    
    print(f"| Validate | loss {loss_metric.compute():.4f} | acc {acc_metric.compute():.4f} |")

In [44]:
#modify as required
def train(
    model: nn.Module, 
    dataloader: DataLoader, 
    optimizer: optim.Optimizer,
    device: torch.device,
    epoch: int,
):
    acc_metric = torchmetrics.Accuracy(task = 'multiclass', num_classes = 10, compute_on_step=False).to(device)
    loss_metric = torchmetrics.MeanMetric(compute_on_step=False).to(device)
    model.train()
    
    # loop through all batches in the training
    for batch in tqdm(dataloader):
        input_ids, input_mask, tags = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        optimizer.zero_grad()
        logits = model(input_ids, input_mask)
        loss = F.cross_entropy(logits.reshape(-1, 10), tags.reshape(-1), ignore_index=0)
        
        loss.backward()
        optimizer.step()
        
        loss_metric.update(loss, input_mask.numel() - input_mask.sum())
        is_active = torch.logical_not(input_mask)
        acc_metric.update(logits[is_active], tags[is_active])
    
    print(f"| Epoch {epoch} | loss {loss_metric.compute():.4f} | acc {acc_metric.compute():.4f} |")
    

In [55]:
#modify as required
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# data loaders
train_dataloader = DataLoader(tr_data, batch_size=32, shuffle=True)
val_dataloader = DataLoader(va_data, batch_size=32)
test_dataloader = DataLoader(te_data, batch_size=32)

# move the model to device
model = TransformerModel(vocab_size = len(tokenizer), 
    embed_size = 256, 
    num_heads = 4, 
    hidden_size = 256,
    num_layers = 2,).to(device)

optimizer = optim.Adam(model.parameters())

for epoch in range(12):
    train(model, train_dataloader, optimizer, device, epoch)
validate(model, val_dataloader, device)

100%|██████████| 439/439 [00:07<00:00, 58.70it/s]


| Epoch 0 | loss 0.5017 | acc 0.8693 |


100%|██████████| 439/439 [00:07<00:00, 57.74it/s]


| Epoch 1 | loss 0.2974 | acc 0.9160 |


100%|██████████| 439/439 [00:07<00:00, 58.91it/s]


...

| Epoch 10 | loss 0.0697 | acc 0.9768 |


100%|██████████| 439/439 [00:07<00:00, 58.43it/s]


| Epoch 11 | loss 0.0639 | acc 0.9791 |


100%|██████████| 102/102 [00:00<00:00, 176.78it/s]

| Validate | loss 0.3307 | acc 0.9296 |


## Todo Part 3
Make predictions on the validation data and evaluate entity-level F1 scores using conlleval script.

`predict`: taking inputs of a trained model, a dataloader, and a torch device, predict the tags for all tokens in the data set. The output should be a nested list of lists, each containing tag predictions for a single sentence.

    Input texts in the dataloader (2 sentences):
    EU rejects German call
    Only France and Britain backed Fischler 's proposal .
    
    Example output:
    [['B-ORG', 'O', 'B-MISC', 'O'], ['O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'O', 'O', 'O']]
        

In [ ]:
# TODO: implement the predict function
# def predict(model: nn.Module, dataloader: DataLoader, device: torch.device) -> List[List[str]]:
#     model.eval()
#     preds = []
#     with torch.no_grad():
#         for batch in tqdm(dataloader):
#             raise NotImplemented
                    
#     return preds



# v1
# def predict(model: nn.Module, dataloader: DataLoader, device: torch.device) -> List[List[str]]:
#     model.eval()
#     preds = []
#     with torch.no_grad():
#         for batch in tqdm(dataloader):
#             batch = [b.to(device) for b in batch]
#             token_ids, mask = batch[:2]
#             logits = model(token_ids, mask)
#             pred_tag_ids = logits.argmax(dim=-1)
#             pred_tags = []
#             for i, tag_ids in enumerate(pred_tag_ids):
#               #trueLen = 
#                 tag_labels = []
#                 for tag_id in tag_ids:
#                     if tag_id == 0:
#                         break
#                     tag_labels.append(NERDataset.idx2tag[tag_id])
#                 pred_tags.append(tag_labels)
#             preds.extend(pred_tags)
#     return preds



def predict(model: nn.Module, dataloader: DataLoader, device: torch.device) -> List[List[str]]:
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            batch = [b.to(device) for b in batch]

            token_ids, mask = batch[0:2]
            if len(batch) != 2:
                tag_ids = batch[2]

            output = model(token_ids, mask)
            pred_tag_ids = output.argmax(dim=-1)
            pred_tags = []
            for i, ids in enumerate(pred_tag_ids):
                trueLen = len(token_ids[i]) - (mask[i] == 1).sum().item()
                tag_labels = [NERDataset.idx2tag[tag_id] for tag_id in ids if tag_id != 0]
                pred_tags.append(tag_labels[:trueLen])
            preds.extend(pred_tags)
    return preds



In [ ]:
!wget https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py
from conlleval import evaluate

--2023-03-16 04:19:42--  https://raw.githubusercontent.com/sighsmile/conlleval/master/conlleval.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7502 (7.3K) [text/plain]
Saving to: ‘conlleval.py’

conlleval.py        100%[===================>]   7.33K  --.-KB/s    in 0s      

2023-03-16 04:19:42 (91.1 MB/s) - ‘conlleval.py’ saved [7502/7502]



In [56]:
# use the conlleval script to measure the entity-level f1
true_tags = []
# trueCurr, predCurr = [],[]
# trueTotal, predTotal = [],[]
pred_tags = []

myPredTags, myTrueTags = [], []

for tags in predict(model, val_dataloader, device):
    pred_tags.extend(tags)
    pred_tags.append('O')
    myPredTags.append(tags)
    # predCurr.extend(tags)
    # predTotal.append(predCurr)


for tags in val_raw['tags']:
    tempTrueTags = tags.strip().split()
    true_tags.extend(tempTrueTags)
    true_tags.append('O')
    myTrueTags.append(tempTrueTags)

    # trueCurr.extend(tags.strip().split())
    # trueTotal.append(trueCurr)
    


100%|██████████| 102/102 [00:18<00:00,  5.57it/s]


In [53]:
num = 56
#for testing equivalency
print(len(myTrueTags[num]), myTrueTags[num])
print(len(myPredTags[num]),myPredTags[num])
wrongCount = 0
for i in range(len(myTrueTags)):
    if(myTrueTags[i] != myPredTags[i]):
        wrongCount += 1
    if(len(myTrueTags[i]) != len(myPredTags[i])):
        print("len", i)
print(wrongCount, len(myTrueTags))


11 ['O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O']
11 ['I-PER', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O']
1584 3250


In [57]:
evaluate(true_tags, pred_tags)

processed 54612 tokens with 5942 phrases; found: 5358 phrases; correct: 3764.
accuracy:  61.66%; (non-O)
accuracy:  93.38%; precision:  70.25%; recall:  63.35%; FB1:  66.62
              LOC: precision:  84.92%; recall:  79.10%; FB1:  81.91  1711
             MISC: precision:  77.30%; recall:  72.02%; FB1:  74.56  859
              ORG: precision:  61.32%; recall:  56.15%; FB1:  58.62  1228
              PER: precision:  57.31%; recall:  48.53%; FB1:  52.56  1560


(70.25009331840238, 63.34567485695052, 66.61946902654867)

Example output from the above codeblock. We will take the overall test F1 score (69.24 in this example) and grade accordingly.
```
processed 54612 tokens with 5942 phrases; found: 5554 phrases; correct: 3980.
accuracy:  65.78%; (non-O)
accuracy:  93.88%; precision:  71.66%; recall:  66.98%; FB1:  69.24
              LOC: precision:  84.58%; recall:  77.03%; FB1:  80.63  1673
             MISC: precision:  77.31%; recall:  71.69%; FB1:  74.40  855
              ORG: precision:  58.71%; recall:  63.83%; FB1:  61.16  1458
              PER: precision:  66.84%; recall:  56.89%; FB1:  61.47  1568
(71.66006481814908, 66.98081454055873, 69.24147529575504)
```
If the codeblock above errors out, check your implementation of the `predict` function. It should return a nested list of lists, each containing predicted tags in their IOB string forms.

## Todo Part 4
Once you finish all previous todos and are satisfied with the model performance on the validation set, make predictions on the test set and keep a copy of the `submission.txt` file by downloading it to your local machine. You can find `submission.txt` under Output > `/kaggle/working`.

In [58]:
# YOU SHOULD NOT CHANGE THIS CODEBLOCK
# make prediction on the test set and save to submission.txt
preds = predict(model, test_dataloader, device)
with open("submission.txt", "w") as f:
    for tags in preds:
        f.write(" ".join(tags) + "\n")

100%|██████████| 108/108 [00:19<00:00,  5.68it/s]


In [ ]:
pwd

'/content'

In [ ]:
ls

190i-mp2-zmarks  conlleval.py.2  __pycache__/    test_tokens.txt
conlleval.py     conlleval.py.3  sample_data/    train.csv
conlleval.py.1   conlleval.py.4  submission.txt  val.csv


In [ ]:
#torch.save(model, "./190i-mp2-zmarks")